In [245]:
# utility functions for accessing transfermarkt.com

from urllib2 import urlopen, Request
from bs4 import BeautifulSoup
from pandas import DataFrame
import re

# transfermarkt blocks default useragent
useragent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
# "lol" can be anything - transfermarkt ignores it/uses it for semantic url
urlprefix = 'http://www.transfermarkt.com/lol/'

def getUrlByLeagueId(id, season=2015):
    return urlprefix + 'startseite/wettbewerb/' + id + '/saison_id/' + str(season)

def getUrlByClubId(id, season=2015):
    return urlprefix + 'startseite/verein/' + str(id) + '/saison_id/' + str(season)

def getUrlByPlayerId(id):
    return urlprefix + 'profil/spieler/' + str(id)

def getUrlByAgentId(id):
    return urlprefix + 'beraterfirma/berater/' + str(id)

def getClubsByLeagueId(id, season=2015):
    bs = BeautifulSoup(urlopen(Request(getUrlByLeagueId(id, season), headers={'User-Agent': useragent})))
    elements = bs.find(id='yw1').find_all("td",class_="hauptlink no-border-links hide-for-small hide-for-pad")
    return [{'clubId': e.find("a")["id"], 'name': e.getText()} for e in elements]

def getPlayersByClubId(id, season=2015):
    bs = BeautifulSoup(urlopen(Request(getUrlByClubId(id, season), headers={'User-Agent': useragent})))
    elements = bs.find(id='yw1').find_all("span",class_="hide-for-small")
    return [{'playerId': e.find("a", class_="spielprofil_tooltip")["id"], 
             'name': e.getText()} for e in elements if e.find("a", class_="spielprofil_tooltip")]

def getTransfersByPlayerId(id):
    bs = BeautifulSoup(urlopen(Request(getUrlByPlayerId(id), headers={'User-Agent': useragent})))
    elements = bs.find(class_="transferhistorie").find_all("tr",class_="zeile-transfer")
    dicts = [{'seasonDate': "  ".join([td.getText() for td in e.findAll("td")[:2]]),
      'mv': e.find("td",class_="zelle-mw").getText(),
      'fee': e.find("td",class_="zelle-abloese").getText(),
      'teams': dict(zip(['from','to'],([{'teamId': team.find("a")["id"], 'name': team.getText()}
                                    for team in e.find_all("td", class_="hauptlink no-border-links hide-for-small vereinsname")])))} for e in elements]
    return [{'season': d['seasonDate'].split("  ")[0],
       'date': d['seasonDate'].split("  ")[1],
       'mv': d['mv'], 'fee': d['fee'],
       'fromTeamId': d['teams']['from']['teamId'],
       'fromTeamName': d['teams']['from']['name'].lstrip(),
       'toTeamId': d['teams']['to']['teamId'],
       'toTeamName': d['teams']['to']['name'].lstrip()} for d in dicts]

def getPlayerData(id):
    bs = BeautifulSoup(urlopen(Request(getUrlByPlayerId(id), headers={'User-Agent': useragent})))
    elements = bs.find(class_="spielerdaten").find_all("tr") 
    result1 = [{'key': e.find("th").getText().strip().rstrip(':'),'value': e.find("td").getText().strip(), 
            'country': (e.find('img',class_="flaggenrahmen").get("title")) if e.find('img',class_="flaggenrahmen") else "",
            'hrefs' : [a.get("href") for a in e.findAll("a")]} for e in elements]

    result2 = [{'key':e['key'], 'value': (e['value'] 
           + (" COUNTRY:" + e['country'] if e['key'] == 'Place of birth' else '')
           + (" HREFs: " + " ".join(e['hrefs']) if len(e['hrefs']) > 0 else '')).strip()
          } for e in result1]

    result3 = dict([(e['key'],e['value']) for e in result2])
    
    result3["Display name"] = bs.find("h1").getText()
    print "now processing " + result3["Display name"].encode('utf-8')
    result3["Date of birth"] = result3["Date of birth"].split("HREFs:")[0].strip()
    result3["Current club id"] = result3["Current club"].split("/verein/")[-1]
    result3["Current club"] = result3["Current club"].split("HREFs:")[0].strip()
    if "Outfitter" in result3:
        result3["Outfitter"] = result3["Outfitter"].split("HREFs:")[0].strip()
    if "Shoe model" in result3:
        result3["Shoe model"] = result3["Shoe model"].split("HREFs:")[0].strip()
    if "Player's agent" in result3:
        result3["Player's agent id"] = result3["Player's agent"].split("/")[-1].strip()
        result3["Player's agent"] = result3["Player's agent"].split("HREFs:")[0].strip()
    if "Place of birth" in result3:
        result3["Country of birth"] = result3["Place of birth"].split("COUNTRY:")[-1].strip()
        result3["Place of birth"] = result3["Place of birth"].split("COUNTRY:")[0].strip()
    result3["Nationality"] = re.sub('\s+', ' ', result3["Nationality"])
    if "on loan from" in result3:
        result3["on loan from club id"] = result3["on loan from"].split("/verein/")[-1].strip()
        result3["on loan from"] = result3["on loan from"].split("HREFs:")[0].strip()
    if "Social media" in result3:    
        socialmedia = dict([(url.split("//")[-1].split("/")[0].split(".")[-2], url) for url in result3["Social media"].split(" ") if url.startswith("http")])    
        for platform in ['twitter', 'facebook', 'instagram']:
            if platform in socialmedia:
                result3[platform]=socialmedia[platform]
        if len([x for x in socialmedia if not x in ['twitter', 'facebook', 'instagram']]) > 0:
            result3['website']=socialmedia[[x for x in socialmedia if not x in ['twitter', 'facebook', 'instagram']][0]]
        del result3["Social media"]    
    
    return result3

In [246]:
getPlayerData(7767)

now processing Fernando Torres


{u'Age': u'32',
 u'Contract there until': u'30.06.2016',
 u'Contract until': u'30.06.2016',
 'Country of birth': u'Spain',
 u'Current club': u'Atl\xe9tico Madrid',
 'Current club id': u'13',
 u'Date of birth': u'Mar 20, 1984',
 'Display name': u'Fernando Torres',
 u'Foot': u'right',
 u'Height': u'1,85 m',
 u'In the team since': u'Jan 6, 2015',
 u'Name in home country': u'Fernando Jos\xe9 Torres Sanz',
 u'Nationality': u'Spain',
 u'Outfitter': u'adidas',
 u'Place of birth': u'Fuenlabrada (Madrid)',
 u"Player's agent": u'Bah\xeda Internacional',
 "Player's agent id": u'1033',
 u'Position': u'Striker - Centre Forward',
 u'Shoe model': u'X 15.1 White Leder since Apr 1, 2016',
 'facebook': u'https://www.facebook.com/Torres/',
 'instagram': u'https://www.instagram.com/fernandotorres/hlde',
 u'on loan from': u'AC Milan',
 'on loan from club id': u'5',
 'twitter': u'https://twitter.com/Torres',
 'website': u'https://www.fernando9torres.com'}

In [220]:
atleti = DataFrame([getPlayerData(player['playerId']) for player in getPlayersByClubId(13)])
atleti

now processing Jan Oblak
now processing Miguel Ángel Moyà
now processing Diego Godín
now processing José Giménez
now processing Stefan Savic
now processing Lucas Hernández
now processing Filipe Luís
now processing Juanfran
now processing Jesús Gámez
now processing Matías Kranevitter
now processing Augusto Fernández
now processing Thomas
now processing Saúl Ñíguez
now processing Gabi
now processing Tiago
now processing Óliver Torres
now processing Koke
now processing Yannick Carrasco
now processing Antoine Griezmann
now processing Ángel Correa
now processing Luciano Vietto
now processing Fernando Torres


,Age,Complete name,Contract option,Contract there until,Contract until,Country of birth,Current club,Current club id,Date of birth,Date of last contract extension,...,Player's agent,Player's agent id,Position,Shoe model,facebook,instagram,on loan from,on loan from club id,twitter,website
0,23,NaN,NaN,NaN,30.06.2021,Slovenia,Atlético Madrid,13,"Jan 7, 1993","Feb 9, 2016",...,NaN,NaN,Goalkeeper,NaN,NaN,oblakjan,NaN,NaN,NaN,NaN
1,32,NaN,NaN,NaN,30.06.2017,Spain,Atlético Madrid,13,"Apr 2, 1984",NaN,...,Manuel García Quilón,792,Goalkeeper,NaN,NaN,m_a_moya,NaN,NaN,M_A_Moya,NaN
2,30,NaN,NaN,NaN,30.06.2019,Uruguay,Atlético Madrid,13,"Feb 16, 1986","Aug 20, 2015",...,corner association,2219,Defence - Centre Back,NaN,diegogodin02,diegogodin,NaN,NaN,diegogodin,NaN
3,21,NaN,NaN,NaN,30.06.2018,Uruguay,Atlético Madrid,13,"Jan 20, 1995",NaN,...,GoalKick Sportmanagement,2563,Defence - Centre Back,NaN,NaN,josemariagimenez13,NaN,NaN,josemagimenez13,NaN
4,25,NaN,NaN,NaN,30.06.2020,Jugoslawien (SFR),Atlético Madrid,13,"Jan 8, 1991",NaN,...,Zarko Pelicic,2901,Defence - Centre Back,NaN,NaN,savkee15,NaN,NaN,NaN,NaN
5,20,NaN,NaN,NaN,30.06.2019,France,Atlético Madrid,13,"Feb 14, 1996","Aug 10, 2015",...,Mioc & Prskalo,2626,Defence - Centre Back,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,30,Filipe Luís Kasmirski,NaN,NaN,30.06.2019,Brazil,Atlético Madrid,13,"Aug 9, 1985",NaN,...,Manuel García Quilón,792,Defence - Left-Back,NaN,filipeluis03_fb_noscript1,filipeluis,NaN,NaN,FilipeLuis03,NaN
7,31,Juan Francisco Torres Belén,NaN,NaN,30.06.2018,Spain,Atlético Madrid,13,"Jan 9, 1985",NaN,...,Toldra Consulting S.L.,3451,Defence - Right-Back,NaN,NaN,NaN,NaN,NaN,Juanfrantorres,NaN
8,31,NaN,NaN,NaN,30.06.2017,Spain,Atlético Madrid,13,"Apr 10, 1985",NaN,...,Manuel García Quilón,792,Defence - Right-Back,NaN,JesusGamez02,jesusgamez02,NaN,NaN,JesusGamez02,NaN
9,22,NaN,NaN,NaN,30.06.2020,Argentina,Atlético Madrid,13,"May 21, 1993",NaN,...,COMUNIGOAL,2143,Midfield - Defensive Midfield,NaN,NaN,mkranevitter5,NaN,NaN,NaN,NaN


In [225]:
getClubsByLeagueId('GB1')

[{'clubId': '281', 'name': u'Manchester City '},
 {'clubId': '631', 'name': u'Chelsea FC '},
 {'clubId': '11', 'name': u'Arsenal FC '},
 {'clubId': '985', 'name': u'Manchester United '},
 {'clubId': '31', 'name': u'Liverpool FC '},
 {'clubId': '148', 'name': u'Tottenham Hotspur '},
 {'clubId': '29', 'name': u'Everton FC '},
 {'clubId': '180', 'name': u'Southampton FC '},
 {'clubId': '762', 'name': u'Newcastle United '},
 {'clubId': '379', 'name': u'West Ham United '},
 {'clubId': '512', 'name': u'Stoke City '},
 {'clubId': '1003', 'name': u'Leicester City '},
 {'clubId': '873', 'name': u'Crystal Palace '},
 {'clubId': '2288', 'name': u'Swansea City '},
 {'clubId': '1010', 'name': u'Watford FC \xa0'},
 {'clubId': '405', 'name': u'Aston Villa '},
 {'clubId': '984', 'name': u'West Bromwich Albion '},
 {'clubId': '1123', 'name': u'Norwich City \xa0'},
 {'clubId': '989', 'name': u'AFC Bournemouth \xa0'},
 {'clubId': '289', 'name': u'Sunderland AFC '}]

In [ ]:
#epl = [getPlayerData(player["playerId"]) for club in getClubsByLeagueId('GB1') for player in getPlayersByClubId(club["clubId"])]
#bundesliga = [getPlayerData(player["playerId"]) for club in getClubsByLeagueId('L1') for player in getPlayersByClubId(club["clubId"])]
#DataFrame(bundesliga).to_excel("bundesliga.xls")

In [221]:
atleti.to_excel("atleti.xls")

In [2]:
DataFrame(getTransfersByPlayerId(121483))

,date,fee,fromTeamId,fromTeamName,mv,season,toTeamId,toTeamName
0,"Jul 16, 2014","16,00 Mill. €",294,Benfica,"4,00 Mill. €",14/15,13,Atlético Madrid
1,"Jun 30, 2013",End of loan,2425,Rio Ave FC,"2,40 Mill. €",12/13,294,Benfica
2,"Jul 1, 2012",Loan,294,Benfica,"1,10 Mill. €",12/13,2425,Rio Ave FC
3,"Jun 30, 2012",End of loan,2639,Leiria,"1,10 Mill. €",11/12,294,Benfica
4,"Aug 1, 2011",Loan,294,Benfica,700 Th. €,11/12,2639,Leiria
5,"Jun 30, 2011",End of loan,4750,Olhanense,750 Th. €,10/11,294,Benfica
6,"Jan 1, 2011",Loan,294,Benfica,750 Th. €,10/11,4750,Olhanense
7,"Dec 31, 2010",End of loan,1436,Beira-Mar,750 Th. €,10/11,294,Benfica
8,"Aug 1, 2010",Loan,294,Benfica,750 Th. €,10/11,1436,Beira-Mar
9,"Jul 1, 2010","1,70 Mill. €",4772,NK Olimpija,750 Th. €,10/11,294,Benfica


In [3]:
DataFrame(getClubsByLeagueId('GB1',2010))  #.sort_values(by="clubId")

,clubId,name
0,631,Chelsea FC
1,985,Manchester United
2,281,Manchester City
3,31,Liverpool FC
4,11,Arsenal FC
5,148,Tottenham Hotspur
6,405,Aston Villa
7,29,Everton FC
8,379,West Ham United
9,289,Sunderland AFC


In [4]:
DataFrame(getPlayersByClubId(418,2005))

,name,playerId
0,Iker Casillas,3979
1,Diego López,34370
2,Jonathan Woodgate,3224
3,Iván Helguera,7514
4,Sergio Ramos,25557
5,Francisco Pavón,7517
6,Álvaro Mejía,16634
7,Roberto Carlos,7518
8,Raúl Bravo,3771
9,Míchel Salgado,7515
